In [0]:
import json
import os
import mlflow
import pandas as pd
import random
from datetime import datetime, timedelta

In [0]:

# Parameters
CATALOG = "mzervou"
SCHEMA = "healthcare"
SOURCE_TABLE = f"{CATALOG}.{SCHEMA}.synthetic_audio_transcripts"
MODEL_NAME = "databricks-claude-3-7-sonnet"

# Step 1: Load the source table
source_df = spark.table(SOURCE_TABLE)

In [0]:
from pyspark.sql.functions import expr

df_transcripts = spark.read.table("mzervou.healthcare.synthetic_audio_transcripts")

df_soap = df_transcripts.withColumn(
    "soap_note",
    expr(
        f"""
        ai_query(
            '{MODEL_NAME}',
            concat(
                'You are a clinical AI assistant. Convert this audio transcript into a structured SOAP note (Subjective, Objective, Assessment, Plan): ',
                audio_transcript
            )
        )
        """
    )
)

df_soap.write.mode("overwrite").saveAsTable("mzervou.healthcare.synthetic_audio_transcripts_enriched")

print("✅ SOAP notes written using ai_query() to table: mzervou.healthcare.synthetic_audio_transcripts_enriched")
